## 와인 종류 분류: 딥러닝을 이용한 다중 클래스 분류

이 노트북은 Scikit-learn에 내장된 와인 데이터셋을 사용하여 세 가지 종류의 와인을 분류하는 딥러닝 모델을 구축합니다. 다중 클래스 분류 문제에 딥러닝을 적용하는 전체 과정을 다룹니다.

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle

### 2. 데이터 로드 및 분할
Scikit-learn의 `load_wine` 함수를 사용하여 데이터를 로드하고, Pandas DataFrame으로 변환하여 내용을 확인합니다. 그 후, 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
def load_data():
    """데이터를 로드하고 훈련/테스트 세트로 분할합니다."""
    wine = load_wine()
    X = pd.DataFrame(wine.data, columns=wine.feature_names)
    y = wine.target # 0, 1, 2 세 종류의 클래스

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=123,
        stratify=y
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_data()

print("--- 데이터 확인 ---")
print("훈련 데이터 형태:", X_train.shape)
print("테스트 데이터 형태:", X_test.shape)
print("\n고유 레이블:", np.unique(y_train))
print("\n첫 5개 훈련 데이터 샘플:")
print(X_train.head())

### 3. 데이터 전처리
딥러닝 모델에 데이터를 입력하기 전에 두 가지 전처리 작업을 수행합니다.

1.  **특성 스케일링**: `StandardScaler`를 사용하여 모든 입력 특성을 표준화합니다.
2.  **레이블 인코딩**: `categorical_crossentropy` 손실 함수를 사용하기 위해, 정수 형태의 레이블(0, 1, 2)을 원-핫 벡터로 변환합니다.

In [ ]:
# 특성 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 레이블 원-핫 인코딩
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

print("--- 전처리 후 데이터 확인 ---")
print("스케일링된 훈련 데이터 형태:", X_train_scaled.shape)
print("원-핫 인코딩된 훈련 레이블 형태:", y_train_encoded.shape)
print("\n첫 번째 원-핫 인코딩된 레이블 (원본: {}):\n{}".format(y_train[0], y_train_encoded[0]))

### 4. 딥러닝 모델 구축
다중 클래스 분류를 위한 `Sequential` 모델을 정의합니다.

- **은닉층**: `relu` 활성화 함수를 사용하는 여러 개의 `Dense` 층으로 구성됩니다.
- **출력층**: 3개의 와인 종류를 분류해야 하므로, 3개의 뉴런과 `softmax` 활성화 함수를 사용합니다. Softmax 함수는 각 클래스에 대한 확률 분포를 출력합니다.

모델 컴파일 시:
- **Optimizer**: `rmsprop`
- **Loss Function**: 다중 클래스 분류 문제이고 레이블이 원-핫 인코딩되었으므로 `categorical_crossentropy`를 사용합니다.
- **Metrics**: `accuracy`를 사용하여 모델 성능을 측정합니다.

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Input(shape=(X_train_scaled.shape[1],)),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(3, activation='softmax'),
    ])

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

### 5. 모델 훈련
모델을 훈련시키면서 `ModelCheckpoint` 콜백을 사용하여 검증 손실(`val_loss`)이 가장 낮은 최적의 모델을 파일로 저장합니다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='와인분류_최적모델.keras',
        save_best_only=True,
        monitor='val_loss'
    )
]

history = model.fit(X_train_scaled, y_train_encoded, 
                    epochs=50, # 에포크 수를 늘려 충분히 학습
                    validation_data=(X_test_scaled, y_test_encoded),
                    callbacks=callbacks, 
                    batch_size=32, # 배치 사이즈 조정
                    verbose=1)

# 훈련 과정 기록을 pickle로 저장
with open("와인분류_훈련기록.pkl", "wb") as file:
    pickle.dump(history.history, file)

### 6. 훈련 과정 시각화
훈련 및 검증 과정에서의 손실과 정확도 변화를 그래프로 그려, 모델의 학습 상태를 확인합니다.

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history)

### 7. 모델 평가
저장된 최적의 모델을 불러와 테스트 데이터셋에 대한 최종 성능을 평가합니다.

In [ ]:
best_model = keras.models.load_model('와인분류_최적모델.keras')

print("--- 최종 모델 평가 (최적 모델) ---")
train_loss, train_acc = best_model.evaluate(X_train_scaled, y_train_encoded)
test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test_encoded)

print(f"훈련셋 손실값: {train_loss:.4f}, 정확도: {train_acc:.4f}")
print(f"테스트셋 손실값: {test_loss:.4f}, 정확도: {test_acc:.4f}")